In [1]:
import openai
import os
from openai import OpenAI
import json
# gpt 请求接口
openai_api_key=""
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
)
# 基于 prompt 生成文本
def get_completion(prompt_sys,prompt_user, model="gpt-4-1106-preview"):
    messages = [
        {"role": "system","content": prompt_sys},
        {"role": "user",  "content": prompt_user}
        ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        # temperature=0.9,  # 模型输出的随机性，0 表示随机性最小
    )
    # client.chat.completions.create
    return response.choices[0].message.content.strip()

In [2]:
prompt_sys="""
你是一名营销文案专家，接下来会获取到``写作要求``与对应生成的``营销文案``
请严格的站在实际应用的角度以及写作要求对营销文案进行评分。5分制,并说明打分原因。
评分维度如下：
```
平台调性（写作要求的平台调性和营销文案所展现的平台调性是否对应）、
品牌信息（写作要求的品牌信息和营销文案的品牌信息是否对应）、
品牌特点/卖点（写作要求的品牌特点/卖点和营销文案的品牌特点/卖点是否对应）、
受众群体（写作要求的受众群体和营销文案的受众群体是否对应）、
可控字数（如果写作要求未提及字数限制，则按照标准社交媒体文案进行评分）、
设定发文者视角（写作要求的设定发文者视角和营销文案的发文者视角是否相同）、
文案标题、
故事性描述（营销文案是否提及一个生动有趣的故事）、
表情/网络用语、
文案通用能力、
文案风格（写作要求的文案风格和营销文案的文案风格是否相同）、
文本生成（营销文案的完整性，连贯性和逻辑性）、
文本创作（营销文案的创新度）、
安全能力
使用场景（营销文案在特定的场景下与写作要求所指定的目标受众的需求和情境相符合）
```
对于任意评分维度，评分标准如下：
```
* 0分：评分维度错误，不明显，无法识别。
* 1分：评分维度比较模糊，但可以勉强识别。
* 2分：评分维度比较清晰，但还有待完善。
* 3分：评分维度比较清晰，并且能够比较准确地传达品牌形象。
* 4分：评分维度非常清晰，并且能够非常准确地传达品牌形象。
* 5分：评分维度非常清晰，并且能够非常准确地传达品牌形象，并且能够让用户产生共鸣。

```
输出要求:返回各个维度的分数与打分理由
输出示例:
    {
        "平台调性":["分数","理由"],
        "品牌信息":["分数","理由"],
        ...
    }

"""


In [4]:
prompt_user = """
写作要求:
```
你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容，使用表情符号等增加文章活力，加入故事性描写来引发读者好奇心
主题：推荐一下[华为HUAWEI]
品牌卖点：阿姨、6寸、快充、外观、5000毫安、外观时尚、电池、电池充电、流畅、6英寸、屏幕、外观好、40w、尺寸、按系统分类、鸿蒙、长辈、60w、系统流畅性、充电功能、充电功率、鸿蒙系统、系统流畅、电池容量
受众群体：[阿姨]
营销目的：吸引客户
```

营销文案:
```
标题：华为HUAWEI，阿姨们的最佳选择！
内容：🌹阿姨们，你们还在为找不到一款适合自己、既时尚又好用的手机而烦恼吗？让我来给你们推荐一款完美的手机——华为HUAWEI！😍🌹华为HUAWEI是一款拥有6英寸屏幕的手机，大小适中，非常适合阿姨们使用。它的屏幕分辨率超高，让画面更加清晰、细腻，无论是看电视剧还是和家人视频通话，都能享受到绝佳的视觉效果。🎁华为HUAWEI还拥有5000毫安大电池，支持40w快充，即使在长时间使用下也能保证电量充足。不再担心手机没电的烦恼，让华为HUAWEI成为你生活中的得力助手。✨它的外观时尚简约，非常符合阿姨们的审美。华为HUAWEI采用了鸿蒙系统，系统流畅性极高，无论是打开应用还是多任务处理，都能轻松应对。而且，它的充电功率高达60w，充电功能也非常强大。🌟总的来说，华为HUAWEI是一款非常优秀的手机。它的6英寸屏幕、大电池容量、时尚外观和流畅的系统都让阿姨们对它爱不释手。如果你也是一位阿姨，想要寻找一款既实用又好用的手机，不妨考虑一下华为HUAWEI吧！🌟🌟🌟
```
"""

result = get_completion(prompt_sys=prompt_sys,prompt_user=prompt_user)
result = json.loads(result)

KeyboardInterrupt: 

In [ ]:
result[0]

In [4]:
prompt_user = """
写作要求:
```
你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容，使用表情符号等增加文章活力，加入故事性描写来引发读者好奇心
主题：推荐一下[华为HUAWEI]
品牌卖点：摄影摄像、品牌、屏幕、年轻人、学生、按系统分类、鸿蒙、华为、屏幕材质、自拍、鸿蒙系统、信号、处理器、e4
受众群体：[学生]
营销目的：提高企业盈利能力
```

营销文案:
```
标题：华为HUAWEI，学生党拍照神器！
内容📸🌟身为一名学生，对于手机的拍照功能总是有着更高的要求。最近，我发现了一款完美的手机——华为HUAWEI，它简直是我拍照的得力助手！😍📸华为HUAWEI的摄影摄像功能真的非常强大。它的镜头组合可以拍摄出各种不同风格的照片和视频，无论是人像、风景还是微距，都能呈现出完美的效果。而且，它的鸿蒙系统也保证了拍摄过程中的流畅性和稳定性。🌟华为HUAWEI的屏幕也是一大亮点。它采用了E4屏幕材质，无论是色彩还原还是亮度表现都非常出色。而且，它的屏幕尺寸也非常适合学生党使用，无论是看视频还是玩游戏，都能享受到绝佳的视觉效果。🌟华为HUAWEI的信号和处理器也是非常优秀的。无论是在校园里还是其他地方，它的信号接收能力都非常强，保证了通讯的稳定性和流畅性。而且，它的处理器性能也非常出色，无论是运行学习软件还是娱乐应用，都能轻松应对。🌟总的来说，华为HUAWEI是一款非常优秀的手机。它的摄影摄像功能、屏幕表现、信号接收能力和处理器性能都让我对它爱不释手。如果你也是一名学生，想要寻找一款既实用又好用的手机，不妨考虑一下华为HUAWEI吧！🌟🌟🌟
```
"""

result = get_completion(prompt_sys=prompt_sys,prompt_user=prompt_user)
print(result)

```json
{
    "平台调性":["5","文案贴合小红书的社区风格和受众偏好，使用了适量表情符号让内容更亲近且有活力。"],
    "品牌信息":["4","品牌信息关于华为品牌的描述较为准确，但未提及具体型号，若要提高辨识度，可加入具体产品型号信息。"],
    "品牌特点/卖点":["5","文案详细阐述了华为HUAWEI的多个卖点，如摄影摄像、鸿蒙系统、E4屏幕材质、信号和处理器等，符合写作要求。"],
    "受众群体":["5","文案充分考虑学生群体的需求，强调了摄影、屏幕质量和性能等对学生十分重要的功能特点。"],
    "可控字数":["5","文案字数适中，为一个适用于小红书平台阅读习惯的篇幅。"],
    "设定发文者视角":["4","明确从学生的视角出发，推荐华为HUAWEI，增加了文案的可信度和针对性。"],
    "文案标题":["5","标题紧扣主题，有吸引力，并且适合学生群体。"],
    "故事性描述":["4","文案通过引入个人使用经验和感受增加了故事性，帮助读者产生共鸣，但可尝试更具体地讲述在学生生活中使用华为HUAWEI的场景。"],
    "表情/网络用语":["5","合理使用了表情符号，增加了文案趣味性，契合小红书用户的阅读和表达习惯。"],
    "文案通用能力":["5","文案具有良好的逻辑结构，语言流畅，信息传递清晰，满足营销文案的通用能力要求。"],
    "文案风格":["5","文案风格适合小红书的受众群体，轻松亲切，同时具备较强说服力。"],
    "文本生成":["5","文本生成能力强，信息完善，结构合理，无明显错误。"],
    "文本创作":["5","文案兼具创意与情感，有效地描绘了产品的特点并能够吸引学生群体。"],
    "安全能力":["5","文案没有出现违规及不当内容，保证了内容的安全性。"]
}
```


In [21]:
result = """
{
    "平台调性":["5","文案贴合小红书的社区风格和受众偏好，使用了适量表情符号让内容更亲近且有活力。"],
    "品牌信息":["4","品牌信息关于华为品牌的描述较为准确，但未提及具体型号，若要提高辨识度，可加入具体产品型号信息。"],
    "品牌特点/卖点":["5","文案详细阐述了华为HUAWEI的多个卖点，如摄影摄像、鸿蒙系统、E4屏幕材质、信号和处理器等，符合写作要求。"],
    "受众群体":["5","文案充分考虑学生群体的需求，强调了摄影、屏幕质量和性能等对学生十分重要的功能特点。"],
    "可控字数":["5","文案字数适中，为一个适用于小红书平台阅读习惯的篇幅。"],
    "设定发文者视角":["4","明确从学生的视角出发，推荐华为HUAWEI，增加了文案的可信度和针对性。"],
    "文案标题":["5","标题紧扣主题，有吸引力，并且适合学生群体。"],
    "故事性描述":["4","文案通过引入个人使用经验和感受增加了故事性，帮助读者产生共鸣，但可尝试更具体地讲述在学生生活中使用华为HUAWEI的场景。"],
    "表情/网络用语":["5","合理使用了表情符号，增加了文案趣味性，契合小红书用户的阅读和表达习惯。"],
    "文案通用能力":["5","文案具有良好的逻辑结构，语言流畅，信息传递清晰，满足营销文案的通用能力要求。"],
    "文案风格":["5","文案风格适合小红书的受众群体，轻松亲切，同时具备较强说服力。"],
    "文本生成":["5","文本生成能力强，信息完善，结构合理，无明显错误。"],
    "文本创作":["5","文案兼具创意与情感，有效地描绘了产品的特点并能够吸引学生群体。"],
    "安全能力":["5","文案没有出现违规及不当内容，保证了内容的安全性。"]
}
"""
result = json.loads(result)
for item in result:
    print(result[item][0])

5
4
5
5
5
4
5
4
5
5
5
5
5
5


In [5]:
prompt_user = """
写作要求:
```
你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容，站在["摄影摄像"]的视角上推荐产品，会使用表情符号等增加文章活力，加入故事性描写来引发读者好奇心
主题：推荐一下[苹果Apple]
品牌卖点：微信、刘海屏、芯片、女性、女朋友、外观、4g运行内存、广角摄像头、摄像、摄影摄像、品牌、苹果、摄像头、按外型分类、打游戏、游戏电竞、镜头、短信通讯、6g运行内存、直屏式、拍照、处理器、摄像头定位、运行内存、颜色
受众群体：[女朋友]
营销目的：吸引客户
```

营销文案:
```
苹果Apple，女友拍照摄影的完美之选！
📸🌟女朋友是个摄影摄像爱好者？还在为找不到一款适合她的手机而烦恼吗？让我来给你们推荐一款完美的手机——苹果Apple！😍📸苹果Apple拥有出色的广角摄像头，可以捕捉到更多的画面细节，让女友在拍照时能够拍出更加广阔的视野。而且，它的摄像头定位也非常准确，无论是拍摄风景还是人像，都能呈现出完美的效果。🌟苹果Apple的拍照功能也非常强大。它的镜头组合可以拍摄出各种不同风格的照片，无论是高清晰度还是低光环境，都能保证照片的质量。而且，它的处理器性能也非常出色，保证了拍摄过程中的流畅性和稳定性。🌟苹果Apple的外观设计也非常精美。它的直屏式外观简洁大方，非常符合女友的审美。而且，它的颜色选择也非常丰富，可以根据女友的喜好来选择最适合的颜色。🌟总的来说，苹果Apple是一款非常优秀的手机。它的摄影摄像功能、外观设计和处理器性能都让女友对它爱不释手。如果你也想为女友寻找一款既实用又好用的手机，不妨考虑一下苹果Apple吧！🌟🌟🌟
```
"""

result = get_completion(prompt_sys=prompt_sys,prompt_user=prompt_user)
print(result)

```json
{
    "平台调性":["5","营销文案很好地贴合了小红书的社区风格，运用了表情符号，增加了文章的活力和生动性，符合小红书年轻用户群体的喜好风格。"],
    "品牌信息":["5","文案清晰地提及了品牌名称——苹果Apple，与写作要求的品牌信息一致。"],
    "品牌特点/卖点":["4","文案较为详细地介绍了苹果手机的摄影摄像功能、外观设计和处理器性能等卖点，但未涉及全部卖点如女性化元素、微信体验等，略有不足。"],
    "受众群体":["5","文案明确指向特定受众群体——女朋友，有利于吸引这一目标受众的关注。"],
    "可控字数":["5","文案长度适中，避免了过长而导致读者产生阅读疲劳，同时又能够包含足够的信息，符合小红书的发文习惯。"],
    "设定发文者视角":["5","文案从摄影摄像爱好者的角度介绍产品，非常贴切写作要求中设定的发文者视角。"],
    "文案标题":["3","虽然文案未显式提供标题，但开头的‘苹果Apple，女友拍照摄影的完美之选！’可以作为一种引人入胜的标题，具有一定的吸引力，但还有改进空间。"],
    "故事性描述":["4","文案通过提问和解答的方式建构了女朋友寻找合适摄影手机的小故事，营造出一定的故事氛围，但故事性可以进一步加强，以更深入地吸引读者。"],
    "表情/网络用语":["5","文案合理使用了表情符号，使内容更加生动，符合小红书的社交媒体语境。"],
    "文案通用能力":["4","文案结构清晰，信息传达准确，语法错误较少，展示了良好的文案通用能力。但在某些部分还可以更加精炼和有力，以增加文案的说服力。"],
    "文案风格":["5","文案采用了小红书用户喜爱的日常亲和风格，与平台调性高度吻合。"],
    "文本生成":["4","文本信息丰富，生成的文案质量较高，适用于社交媒体营销，但在某些细节描述上仍有加强的空间，例如可以更具体地陈述某些功能的优势。"],
    "文本创作":["4","文案在创作过程中很好地融入了故事元素和情感因素，能够引起受众的共鸣，但仍有进一步优化和创新的空间。"],
    "安全能力":["5","文案在内容上没有任何违规或不当之处，遵守了营销道德和社交平台规则，显示出良好的安全能力。"]
}


In [11]:
prompt_user = """
写作要求:
```
你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容，站在婚庆摄影的视角上推荐产品，会使用表情符号等增加文章活力，加入故事性描写来引发读者好奇心，在文案中加入相关标签或话题来突出重点内容。
主题：推荐一下小米Civi 3
品牌卖点：信号好，通话质量高，适合在嘈杂环境下使用，全新设计风格，超轻薄便携。
字数要求：300字左右
受众群体：无人机操作员
使用场景：婚礼现场、工作室、车上
营销目的：提高品牌知名度
文案风格：简洁明了风格
```

营销文案:
```
🚀 小米Civi 3：为婚庆摄影打造的通信神器✨

🌟 在婚庆摄影中，与团队的即时沟通是关键。小米Civi 3以其卓越的信号质量和通话体验，确保每一刻的完美捕捉。不论是在繁华的城市街头，还是在偏远的乡村，它都能为你提供稳定的通信保障。

💖 全新设计风格，超轻薄便携，是婚庆摄影团队的理想选择。无论是忙碌的婚礼现场，还是工作室的紧张筹备，甚至在车上抢拍美好瞬间，它都能轻松伴你左右。

🔥 选择小米Civi 3，选择为婚庆摄影带来前所未有的便利和品质。在未来的每一场婚礼中，让我们一同见证这份美好。#小米Civi 3 #婚庆摄影 #通信保障
```
"""

result = get_completion(prompt_sys=prompt_sys,prompt_user=prompt_user)
print(result)

{
    "平台调性":["4","文案采用了营养表情符号，语言简洁，适合小红书年轻和追求个性的用户群体"],
    "品牌信息":["4","文案围绕小米Civi 3手机进行描述，提到了品牌和产品名称"],
    "品牌特点/卖点":["5","文案准确阐述了小米Civi 3的信号好、通话质量高、全新设计风格、超轻薄便携等卖点"],
    "受众群体":["3","根据受众群体是无人机操作员，文案与无人机操作、婚礼现场和车上工作的使用场景相对应，但没有特别强调无人机操作员的特殊需求"],
    "可控字数":["5","文案字数控制在了300字左右，符合写作要求"],
    "设定发文者视角":["4","文案成功从婚庆摄影的视角出发，推荐产品，视角符合要求"],
    "文案标题":["4","标题吸引注意力，并且表达了文案的主旨"],
    "故事性描述":["5","运用了婚礼的场景和对话体验的故事化描述增加了文案的吸引力"],
    "表情/网络用语":["5","文案运用了表情符号，符合小红书的使用习惯，让文案更生动活泼"],
    "文案通用能力":["4","文案整体结构清晰，信息传达准确，语言表达简洁明了，具有较好的通用营销文案能力"],
    "文案风格":["3","文案风格偏向简洁明了，但部分表达可能在小红书平台上显得不够轻松诙谐"],
    "文本生成":["0","这个维度不适用于手动评分，通常是自动生成文本的评价指标"],
    "文本创作":["5","文案具有创造性和新颖性，能够吸引目标受众"],
    "安全能力":["5","文案内容健康，没有违反任何广告法规定或敏感话题，安全合规"]
}


In [7]:
prompt_user = """
写作要求:
```
帮我生成一下小米MIX FOLD 2营销文案，小红书文风，站在工作者的视角上推荐产品，用表情符号等增加文章活力，加入故事性描写，品牌卖点：超长待机时间，快速文件传输，超长电池寿命，信号优良。字数要求：300字左右。受众群体：出差需求较多的白领。使用场景：出差途中。营销目的：提高品牌知名度和销售额。文案风格：权威专业风格，在文案中加入相关标签或话题来突出重点内容。
```

营销文案:
```
标题: 小米 MIX FOLD 2,出差必备神器！
内容:🔋【小米 MIX FOLD 2】，让你出差工作无忧的神器！📱
🌟【超长待机】: 出差途中，信号不好，电池续航是你的安全感。🔋
⚡【快速文件传输】: 手机一碰，立刻秒传，让你的工作效率翻倍。🚀
🌟【超长电池寿命】: 告别电量焦虑，出差过程中轻松应对各种意外情况。🔋
🚀【信号优良】: 出差途中，信号不好？小米 MIX FOLD 给你带来稳定的网络连接。📶
🎉小仙女们，快来体验小米 MIX FOLD 2,让你的出差工作变得更加便捷和高效！🚀
#小米 MIX FOLD #出差神器#超长待机#快速传输#电池续航#信号增强
```
"""

result = get_completion(prompt_sys=prompt_sys,prompt_user=prompt_user)
print(result)

```json
{
    "平台调性":["3","文案成功地运用了小红书的文风，通过使用表情符号和活泼的语言增加文章活力。"],
    "品牌信息":["4","品牌信息非常清晰，‘小米 MIX FOLD 2’被重复提及，确保了品牌信息的传达。"],
    "品牌特点/卖点":["3","文案对产品的卖点进行了描述，但是描述方式略显简单，能够进一步增加详情以加强描述的说服力。"],
    "受众群体":["4","明确针对‘出差需求较多的白领’，这个受众群体的诉求进行了专门的描述。"],
    "可控字数":["3","文案字数适中，且信息较为紧凑，未超出规定字数限制。"],
    "设定发文者视角":["4","站在工作者的视角推荐产品的要求得到了满足，但可通过增加具体的故事情节来进一步突出这一视角。"],
    "文案标题":["4","标题简洁有力，紧扣产品特色与使用场景。"],
    "故事性描述":["2","文案涵盖了故事性描述，但并不充分。描述过于直接，缺乏将读者代入具体场景的故事性构建。"],
    "表情/网络用语":["5","充分运用了表情符号，为文案增添活力并适应小红书的平台风格。"],
    "文案通用能力":["3","文案通用能力良好，能够传达关键信息，但仍有提升空间，如通过更具体的描述和故事来增强说服力。"],
    "文案风格":["3","文案尝试结合权威专业和小红书文风，但权威专业风格还不够显著，可以加入更多专业术语或数据支撑。"],
    "文本生成":["4","文案顺畅且没有语法或拼写错误，文本生成的质量较高。"],
    "文本创作":["3","创作思路明确，重点突出，但在故事性描写方面可以更深入，以引起用户的共鸣。"],
    "安全能力":["5","文案内容适当，无不妥言论或敏感信息，显示出良好的内容审查能力。"]
}
```


In [8]:
prompt_user = """
写作要求:
```
你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容，站在实用主义的视角上推荐产品，会使用表情符号等增加文章活力，加入故事性描写来引发读者好奇心，在文案中加入相关标签或话题来突出重点内容。
主题：推荐一下小米12T Pro
品牌卖点：耐用性好、展示效果优秀、较强的人机互动、功能齐全、便携、易于充电，数据安全性高
受众群体：人工智能研究员
使用场景：展览会现场、研究院办公室、会议现场
营销目的：树立品牌形象
文案风格：亲切友好风格
```

营销文案:
```
标题: 小米12T Pro: 超越想象的智能助手，助您畅游数字世界！
内容:🎉亲们，今天给大家安利一款超级给力的神器——小米12T Pro!📱
👀这款手机可是 AI 界的翘楚，特别适合我们这些人工智能研究员使用。它不仅展示效果一流，人机互动也是一绝，让你的工作效率翻倍！🚀
👍该机的耐用性和稳定性都非常出色，让你的工作更加无忧。🌟
📸该机配备了摄像头，让你拍摄照片更加清晰。而且，它还拥有漫长的待机时间，让你在外出时无需担心电量不足。🔋
🔒此外，它还具备防盗保藏的特点，让你在展会现场时不用担心手机被盗。🛡️
🚀小米12T Pro 不仅仅是一部手机，更是你工作效率提升的利器。无论是在展会现场、办公室还是会议现场，它都能让你的工作更加便捷。🎓
💖如果你也想体验这款强大的手机，就赶紧种草吧！🌱
#小米12T Pro #人工智能研究员#工作利器#手机#展会#办公室#会议现场🌟🎉📱🔒📸🛡️🚀
```
"""

result = get_completion(prompt_sys=prompt_sys,prompt_user=prompt_user)
print(result)

```json
{
    "平台调性": ["3", "制作了符合小红书平台的营销文案，使用了表情符号来增加活力但能够再富有个性一些"],
    "品牌信息": ["3", "提及小米12T Pro，但没有详细展开品牌背景或历史"],
    "品牌特点/卖点": ["4", "很好的提及了耐用性、展示效果、人机互动、摄像头、待机时间以及数据安全等卖点"],
    "受众群体": ["3", "文案提及受众为人工智能研究员，但是没有深入解释产品怎么满足这一受众群体的专业需求"],
    "可控字数": ["3", "文案长度适中，信息较为完整，但还可以更精炼"],
    "设定发文者视角": ["3", "从一个推荐者的视角来介绍产品，适合小红书平台"],
    "文案标题": ["4", "标题吸引人，符合小红书的文风，也很好地传达了产品卖点"],
    "故事性描述": ["3", "文案中加入了一些故事化元素，如展会现场、办公室等使用场景，但可以更具体地描述使用过程中的情景"],
    "表情/网络用语": ["4", "文案中使用了不少表情符号，增加了阅读的趣味性，符合平台风格"],
    "文案通用能力": ["3", "文案结构合理，但在叙事连接和逻辑性方面还有提升空间"],
    "文案风格": ["3", "整体风格亲切友好，但并没有突出一些更具个性化的特点"],
    "文本生成": ["3", "文案内容合适但没有具体描述象操作界面，交互设计细节等方面的内容"],
    "文本创作": ["3", "创作内容符合要求，能够吸引目标受众，但创意层面还有提升空间"],
    "安全能力": ["5", "文案没有包含任何不安全或敏感的内容"]
}
```


In [14]:
prompt_user = """
写作要求:
```
你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容，站在实用主义的视角上推荐产品，会使用表情符号等增加文章活力，加入故事性描写来引发读者好奇心，在文案中加入相关标签或话题来突出重点内容。
主题：推荐一下小米12T Pro
品牌卖点：耐用性好、展示效果优秀、较强的人机互动、功能齐全、便携、易于充电，数据安全性高
受众群体：人工智能研究员
使用场景：展览会现场、研究院办公室、会议现场
营销目的：树立品牌形象
文案风格：亲切友好风格
```

营销文案:
```
标题：华为HUAWEI，阿姨们的最佳选择！
内容：🌹阿姨们，你们还在为找不到一款适合自己、既时尚又好用的手机而烦恼吗？让我来给你们推荐一款完美的手机——华为HUAWEI！😍🌹华为HUAWEI是一款拥有6英寸屏幕的手机，大小适中，非常适合阿姨们使用。它的屏幕分辨率超高，让画面更加清晰、细腻，无论是看电视剧还是和家人视频通话，都能享受到绝佳的视觉效果。🎁华为HUAWEI还拥有5000毫安大电池，支持40w快充，即使在长时间使用下也能保证电量充足。不再担心手机没电的烦恼，让华为HUAWEI成为你生活中的得力助手。✨它的外观时尚简约，非常符合阿姨们的审美。华为HUAWEI采用了鸿蒙系统，系统流畅性极高，无论是打开应用还是多任务处理，都能轻松应对。而且，它的充电功率高达60w，充电功能也非常强大。🌟总的来说，华为HUAWEI是一款非常优秀的手机。它的6英寸屏幕、大电池容量、时尚外观和流畅的系统都让阿姨们对它爱不释手。如果你也是一位阿姨，想要寻找一款既实用又好用的手机，不妨考虑一下华为HUAWEI吧！🌟🌟🌟

```
"""

result = get_completion(prompt_sys=prompt_sys,prompt_user=prompt_user)
print(result)

```json
{
    "平台调性":["0","文案与小红书平台调性严重不符，提到的华为品牌与要求的小米品牌不一致，且使用了与要求不符的语气和表情，缺乏小红书文风所特有的生活化和亲和力。"],
    "品牌信息":["0","文案错误地将品牌华为作为主题，与要求推广小米12T Pro的品牌信息不相符。"],
    "品牌特点/卖点":["0","文案中完全没有提及小米12T Pro的任何卖点，而是错误介绍了华为手机的特点，与写作要求不匹配。"],
    "受众群体":["0","文案以阿姨作为目标受众群体，这与写作要求中明确指出的目标受众群体——人工智能研究员不符。"],
    "可控字数":["3","文案的长度在一定程度上适合社交媒体平台，虽然具体字数未提及，但长度合理不过分冗长。"],
    "设定发文者视角":["0","文案中发文者视角与要求完全不符，要求站在实用主义视角推荐产品，文案却站在普通人的视角，为阿姨群体推荐。"],
    "文案标题":["0","标题与写作要求的主题完全不符，提到了华为而不是小米12T Pro。"],
    "故事性描述":["0","文案没有体现出任何故事性描述，没有创造场景或背景来吸引读者，也没有遵从写作要求中关于增加故事性描写的指示。"],
    "表情/网络用语":["0","虽然文案使用了表情，但是其使用并没有增加文章的活力，也未能根据写作要求有效地增强营销文案的吸引力。"],
    "文案通用能力":["1","文案具有最基础的叙述能力，能够描述一个品牌和产品的特征，尽管这与要求的产品不符。"],
    "文案风格":["0","文案的风格与所要求的亲切友好风格相违背，出现误导并且与实际要求的产品和受众不相符。"],
    "文本生成":["1","文案的结构具有基本的连贯性，但由于错误的品牌和受众，其整体逻辑和完整性受到了严重影响。"],
    "文本创作":["0","文案没有体现出任何创新元素，未遵循写作要求，创意度极低。"],
    "安全能力":["3","尽管文案和品牌信息错误，但文案的内容在安全性方面没有明显问题，没有不当言论或可能导致负面社会影响的表述。"]
}
```
